# Q-learning 

### Group Members: Bhavesh Ajwani (20102409), Kondury Rishabh (201020425)

# The Environment

### The first task is defining the system 
#### * States : The state of the system is the row and column number of the grid, terminal states are the walls in the grid
#### * Actions : The defined actions are up, down, left, right 
#### * Rewards : For the movable sides, the reward is -1, -100 for walls and +100 for ending position.

### Goal : The goal is to find the shortest path from the start to the end poisiton in the grid.

In [1]:
#importing required libraries
import numpy as np
import pandas as pd
import random

In [2]:
#taking user input for the size of the grid:
n = int(input("Enter the size of the grid: "))
rows = columns = n;

#taking input for starting coordinates 
sr, sc = input("Enter the coordinates of starting position: ").split()
sr = int(sr)
sc = int(sc)

Enter the size of the grid: 6
Enter the coordinates of starting position: 2 3


In [3]:
#making q-table for the matrix by initialising with zeroes
q_table = np.zeros((rows, columns, 4))
q_table

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [4]:
#defining actions 
actions = ['left', 'right', 'down', 'up']

### The building of reward matrix is done by selecting the obstacles in a randomized order. The 10% of total cells are coloured black and others are reachable.

In [5]:
#builiding the required grid: 
rewards = np.full((n, n), -100.0)

#filling the 90% part of the grid with -1
ninety_percent = (int)(n*n*0.90)

lst = random.sample(range(0, n*n), ninety_percent)

movable_cells = []
for i in lst:
    movable_cells.append([i//n, i%n])

for i in movable_cells:
    rewards[i[0]][i[1]] = -1.0

#for ending position:
rewards[sr, sc] = 100.0

### Reward matrix: 

In [6]:
re_table = pd.DataFrame(rewards)
display(re_table)

,0,1,2,3,4,5
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-100.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,100.0,-1.0,-1.0
3,-100.0,-1.0,-100.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-100.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [7]:
#this function checks if the current state is a terminal state or not
def is_terminal_state(current_r, current_c):
    if rewards[current_r, current_c] == -1.0:
        return False
    else:
        return True

#while the next state is terminal state, the random values are checked and non-terminal states are found.
def get_start():
    current_r = np.random.randint(rows)
    current_c = np.random.randint(columns)


    while is_terminal_state(current_r, current_c):
        current_r = np.random.randint(rows)
        current_c = np.random.randint(columns)

    return current_r, current_c

#The next action from the current state is defined 
def get_next_action(current_r, current_c, ep):
    if np.random.random() < ep:
        return np.argmax(q_table[current_r, current_c])
    else:
        return np.random.randint(4)

#Based on the action, the next state is being selected in the greedy-epsilon method
def get_next_location(current_r, current_c, action_idx):
    next_r = current_r
    next_c = current_c

    if actions[action_idx] == 'up' and current_r > 0:
        next_r -= 1
    elif actions[action_idx] == 'right' and current_c < columns-1:
        next_c += 1
    elif actions[action_idx] == 'down' and current_r < rows-1:
        next_r += 1
    elif actions[action_idx] == 'left' and current_c > 0:
        next_c -= 1
    
    return next_r, next_c

#The shortest path is found from the state : (r, c)
def get_shortest_path(r, c):
    if is_terminal_state(r, c):
        return []
    else:
        current_r, current_c = r, c
        shortest_path = []
        shortest_path.append([current_r, current_c])

        while not is_terminal_state(current_r, current_c):
            action_idx = get_next_action(current_r, current_c, 1.0)

            current_r, current_c = get_next_location(current_r, current_c, action_idx) 

            shortest_path.append([current_r, current_c])

        return shortest_path

## The training of the model is done by selecting the hyperparameters: 

In [9]:
ep = 0.9 #expliotation and exploration ratio 
alpha = 0.9 #learning rate
gamma = 0.9 #discount factor
episodes = 500 #the number of episodes 


for it in range(episodes):
    row, col = get_start()


    while not is_terminal_state(row, col):
        action_idx = get_next_action(row, col, ep)

        old_r, old_c = row, col
        row, col = get_next_location(row, col, action_idx)

        reward = rewards[row, col]
        old_q_value = q_table[old_r, old_c, action_idx]
        temp_diff = reward + (gamma * np.max(q_table[row, col])) - old_q_value

        new_q_value = old_q_value + (alpha * temp_diff)
        q_table[old_r, old_c, action_idx] = new_q_value


print("Finished")

Finished


In [10]:
#Printing the shortest path from the given vertices:
print(get_shortest_path(4, 0))

[[4, 0], [4, 1], [3, 1], [2, 1], [2, 2], [2, 3]]


## Q-table after training:

In [11]:
q_table

array([[[  -3.1425039 ,   62.17099932,  -90.        ,   -3.1425039 ],
        [  -3.0688749 ,   -2.39958   ,   70.19      ,   55.707471  ],
        [  -1.728     ,   -1.728     ,   79.1       ,   -1.719     ],
        [  -1.719     ,   62.92832817,   89.        ,   -0.9       ],
        [  79.09999913,   -0.99      ,   -1.719     ,   -1.719     ],
        [  -1.719     ,   -1.719     ,   70.19      ,   -0.9       ]],

       [[   0.        ,    0.        ,    0.        ,    0.        ],
        [ -99.        ,   78.29180906,   79.1       ,   60.89212434],
        [  56.53293851,   89.        ,   80.01      ,   69.47256094],
        [  71.1       ,   79.09192717,  100.        ,    0.        ],
        [  89.        ,   63.081     ,   -0.9       ,   -0.9       ],
        [  -0.99      ,   63.01475075,   79.1       ,    0.        ]],

       [[  -1.719     ,   79.1       ,  -90.        ,  -90.        ],
        [  70.18992233,   89.        ,   70.11891   ,   70.18999993],
        [  71.1 